# Trung Nguyen - 111752939
## Assignment #3: Data Integration and Modeling 

## Task 1 (10%)

> - Build a scoring function to rank houses by “desirability”, presumably a notion related to cost or value.  
- Identify what the ten most desirable and least desirable houses in the Kaggle data set are.
- Write a one page description of which variables your function used and how well you think it worked. 

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib
import math
from sklearn import datasets, linear_model, preprocessing
from sklearn.metrics import mean_squared_error, r2_score, pairwise
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import f_regression
import warnings
warnings.filterwarnings('ignore')

df = pd.read_csv('./data/properties_2017.csv', low_memory=False)
# Calculate mean value and replace NaN values with mean value

There are fields whose values are numercal but they should be listed as categorical, since their numerical values does not reflex the level of affection to the model (Such as ``typeid`` fields). Changing each of their value to a new binary feature would be the right way to extract information from them, but then the model becomes very complex with too many features, so I simply dropped them.

I also dropped all binary fields, as I don't have experience with handling both categorical features and numerical features in the same model.

I also dropped all fields whose whose the ratio of NaN values > 25%

Finally, I replaced all NaN values in  whose 

In [2]:
df_dropped = df.drop(["hashottuborspa", "propertycountylandusecode", "propertyzoningdesc", 
                            "fireplaceflag", "taxdelinquencyflag", "assessmentyear", 
                            "airconditioningtypeid", "buildingclasstypeid", "decktypeid",
                            "heatingorsystemtypeid", "regionidcity", "regionidcounty",
                            "poolcnt", "fips", "threequarterbathnbr",
                            "regionidneighborhood", "regionidzip", "storytypeid",
                            "pooltypeid7", "rawcensustractandblock", "censustractandblock",
                            "typeconstructiontypeid", "pooltypeid10", "pooltypeid2", 
                            "architecturalstyletypeid", "bathroomcnt", "propertylandusetypeid"], axis=1)
# df_dropped_filled0 = df_dropped.fillna(0)
row, column = df_dropped.shape

In [3]:
# Drop all columns whose the ratio of NaN values > 50%
df_dropped1 = df_dropped.copy()
row, column = df_dropped1.shape
for x in df_dropped1:
    if df_dropped1[str(x)].isnull().sum() > row*0.5:
        df_dropped1.drop(str(x), axis=1, inplace=True)

In [4]:
for x in df_dropped1:
    # 'np.nanmean' finds mean by ignoring NaN values, we then replace NaN values with the result
    median = np.nanmedian(df_dropped1[str(x)])
    df_dropped1[str(x)].fillna(value=median, inplace=True)
#     for i,y in enumerate(df_dropped1[str(x)]):
#         if y=='nan' or y=="nan" or math.isnan(y):
#             df_dropped1[str(x)][i] = median

In [5]:
df_dropped1.head()

,parcelid,bedroomcnt,buildingqualitytypeid,calculatedbathnbr,calculatedfinishedsquarefeet,finishedsquarefeet12,fullbathcnt,latitude,longitude,lotsizesquarefeet,roomcnt,unitcnt,yearbuilt,structuretaxvaluedollarcnt,taxvaluedollarcnt,landtaxvaluedollarcnt,taxamount
0,10754147,0.0,6.0,2.0,1574.0,1542.0,2.0,34144442.0,-118654084.0,85768.0,0.0,1.0,1963.0,127066.0,9.0,9.0,4007.62
1,10759547,0.0,6.0,2.0,1574.0,1542.0,2.0,34140430.0,-118625364.0,4083.0,0.0,1.0,1963.0,127066.0,27516.0,27516.0,4007.62
2,10843547,0.0,6.0,2.0,73026.0,1542.0,2.0,33989359.0,-118394633.0,63085.0,0.0,2.0,1959.0,660680.0,1434941.0,774261.0,20800.37
3,10859147,0.0,6.0,2.0,5068.0,1542.0,2.0,34148863.0,-118437206.0,7521.0,0.0,1.0,1948.0,580059.0,1174475.0,594416.0,14557.57
4,10879947,0.0,6.0,2.0,1776.0,1542.0,2.0,34194168.0,-118385816.0,8512.0,0.0,1.0,1947.0,196751.0,440101.0,243350.0,5725.17


In [6]:
print(df_dropped1.shape)
print(df_dropped.shape)
df_dropped1.describe()

(2985217, 17)
(2985217, 31)


,parcelid,bedroomcnt,buildingqualitytypeid,calculatedbathnbr,calculatedfinishedsquarefeet,finishedsquarefeet12,fullbathcnt,latitude,longitude,lotsizesquarefeet,roomcnt,unitcnt,yearbuilt,structuretaxvaluedollarcnt,taxvaluedollarcnt,landtaxvaluedollarcnt,taxamount
count,2.985217e+06,2.985217e+06,2.985217e+06,2.985217e+06,2.985217e+06,2.985217e+06,2.985217e+06,2.985217e+06,2.985217e+06,2.985217e+06,2.985217e+06,2.985217e+06,2.985217e+06,2.985217e+06,2.985217e+06,2.985217e+06,2.985217e+06
mean,1.332586e+07,3.093270e+00,6.182629e+00,2.291902e+00,1.827566e+03,1.744376e+03,2.238353e+00,3.400131e+07,-1.182016e+08,2.117832e+04,1.472891e+00,1.119652e+00,1.964415e+03,1.773479e+05,4.421233e+05,2.666122e+05,5.398269e+03
std,7.909966e+06,1.270945e+00,1.401481e+00,9.851282e-01,1.939635e+03,9.866635e+02,9.745070e-01,2.433962e+05,3.454233e+05,2.383343e+05,2.838837e+00,2.027726e+00,2.345478e+01,4.564998e+05,8.117426e+05,4.817740e+05,9.639395e+03
min,1.071172e+07,0.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,3.332439e+07,-1.194758e+08,1.000000e+02,0.000000e+00,1.000000e+00,1.801000e+03,1.000000e+00,1.000000e+00,1.000000e+00,2.400000e-01
25%,1.164371e+07,2.000000e+00,6.000000e+00,2.000000e+00,1.220000e+03,1.229000e+03,2.000000e+00,3.382756e+07,-1.183926e+08,5.866000e+03,0.000000e+00,1.000000e+00,1.950000e+03,7.839700e+04,1.897140e+05,8.178200e+04,2.480880e+03
50%,1.254509e+07,3.000000e+00,6.000000e+00,2.000000e+00,1.574000e+03,1.542000e+03,2.000000e+00,3.400807e+07,-1.181721e+08,7.000000e+03,0.000000e+00,1.000000e+00,1.963000e+03,1.270660e+05,3.211610e+05,1.766190e+05,4.007620e+03
75%,1.409712e+07,4.000000e+00,7.000000e+00,3.000000e+00,2.127000e+03,2.000000e+03,3.000000e+00,3.416171e+07,-1.179491e+08,9.222000e+03,0.000000e+00,1.000000e+00,1.981000e+03,2.022470e+05,5.104360e+05,3.219020e+05,6.204640e+03
max,1.696019e+08,2.500000e+01,1.200000e+01,3.200000e+01,9.525760e+05,4.270790e+05,3.200000e+01,3.481965e+07,-1.175543e+08,3.710005e+08,9.600000e+01,9.970000e+02,2.016000e+03,2.553212e+08,3.196225e+08,9.401108e+07,3.823176e+06


In [7]:
#Reference: http://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.minmax_scale.html

#Scaling all variables to a range of (0,cap), where cap = 10000
#This is done by: X_std = (X - X.min) / (X.max - X.min)
#                 X_scaled = X_std * (max - min) + min
#Using an API referred above to do the computation
df_dropped1_scaled = df_dropped1.copy()
cap = 100
df_dropped1_scaled["bedroomcnt"] = preprocessing.minmax_scale(df_dropped1["bedroomcnt"], (0,cap))
df_dropped1_scaled["buildingqualitytypeid"] = preprocessing.minmax_scale(df_dropped1["buildingqualitytypeid"], (0,cap))
df_dropped1_scaled["calculatedbathnbr"] = preprocessing.minmax_scale(df_dropped1["calculatedbathnbr"], (0,cap))
df_dropped1_scaled["calculatedfinishedsquarefeet"] = preprocessing.minmax_scale(df_dropped1["calculatedfinishedsquarefeet"], (0,cap))
df_dropped1_scaled["finishedsquarefeet12"] = preprocessing.minmax_scale(df_dropped1["finishedsquarefeet12"], (0,cap))
df_dropped1_scaled["fullbathcnt"] = preprocessing.minmax_scale(df_dropped1["fullbathcnt"], (0,cap))
df_dropped1_scaled["lotsizesquarefeet"] = preprocessing.minmax_scale(df_dropped1["lotsizesquarefeet"], (0,cap))
df_dropped1_scaled["roomcnt"] = preprocessing.minmax_scale(df_dropped1["roomcnt"], (0,cap))
df_dropped1_scaled["unitcnt"] = preprocessing.minmax_scale(df_dropped1["unitcnt"], (0,cap))
df_dropped1_scaled["yearbuilt"] = preprocessing.minmax_scale(df_dropped1["yearbuilt"], (0,cap))
df_dropped1_scaled["structuretaxvaluedollarcnt"] = preprocessing.minmax_scale(df_dropped1["structuretaxvaluedollarcnt"], (0,cap))
df_dropped1_scaled["taxvaluedollarcnt"] = preprocessing.minmax_scale(df_dropped1["taxvaluedollarcnt"], (0,cap))
df_dropped1_scaled["landtaxvaluedollarcnt"] = preprocessing.minmax_scale(df_dropped1["landtaxvaluedollarcnt"], (0,cap))
df_dropped1_scaled["taxamount"] = preprocessing.minmax_scale(df_dropped1["taxamount"], (0,cap))

In [8]:
field_no = df_dropped1_scaled.shape[1]

rank =  df_dropped1_scaled["bedroomcnt"]\
        - df_dropped1_scaled["buildingqualitytypeid"] + \
        df_dropped1_scaled["calculatedbathnbr"] + \
        df_dropped1_scaled["calculatedfinishedsquarefeet"] + \
        df_dropped1_scaled["finishedsquarefeet12"] + \
        df_dropped1_scaled["fullbathcnt"] + \
        df_dropped1_scaled["lotsizesquarefeet"] + \
        df_dropped1_scaled["roomcnt"] + \
        df_dropped1_scaled["unitcnt"]\
        - df_dropped1_scaled["yearbuilt"] + \
        df_dropped1_scaled["structuretaxvaluedollarcnt"] + \
        df_dropped1_scaled["taxvaluedollarcnt"] + \
        df_dropped1_scaled["landtaxvaluedollarcnt"] + \
        df_dropped1_scaled["taxamount"]

mostAbove = df_dropped1.loc[rank.argmax()]
mostBelow = df_dropped1.loc[rank.argmin()]
# print("House ID with highest Rank: ",mostAbove)
# print("House ID with lowest Rank: ",mostBelow)

In [9]:
df_dropped1_rank = df_dropped1.copy()
df_dropped1_rank['Rank'] = rank.values
df_dropped1_sorted = df_dropped1_rank.sort_values(by='Rank',axis=0,ascending=False)

In [10]:
df_dropped1_sorted.nlargest(10, 'Rank')

,parcelid,bedroomcnt,buildingqualitytypeid,calculatedbathnbr,calculatedfinishedsquarefeet,finishedsquarefeet12,fullbathcnt,latitude,longitude,lotsizesquarefeet,roomcnt,unitcnt,yearbuilt,structuretaxvaluedollarcnt,taxvaluedollarcnt,landtaxvaluedollarcnt,taxamount,Rank
342941,11746970,0.0,6.0,2.0,520825.0,1542.0,2.0,34061216.0,-118290117.0,7000.0,0.0,464.0,2013.0,255321161.0,287098486.0,31777325.0,3458861.12,278.013245
924829,167688009,0.0,6.0,2.0,1574.0,1542.0,2.0,34050073.0,-118259422.0,7000.0,0.0,1.0,1963.0,228000000.0,319622473.0,91622473.0,3823175.65,272.934571
1252741,11984395,0.0,6.0,2.0,472363.0,1542.0,2.0,34061321.0,-118342823.0,7000.0,0.0,478.0,2014.0,222334475.0,271004605.0,48670130.0,3277055.20,269.124903
2241598,11645223,7.0,6.0,20.0,28725.0,28725.0,20.0,34080136.0,-118443187.0,365955.0,0.0,1.0,1938.0,24469437.0,83196095.0,58726658.0,994030.96,175.326106
1312470,11783635,0.0,6.0,2.0,1574.0,1542.0,2.0,34046144.0,-118255035.0,7000.0,0.0,1.0,1963.0,157500000.0,224345565.0,66845565.0,2772707.63,161.681457
1316928,11641900,12.0,6.0,15.0,35046.0,35046.0,15.0,34085652.0,-118427058.0,182203.0,0.0,1.0,2002.0,10355346.0,88020448.0,77665102.0,1052070.50,153.039118
2296330,11645396,10.0,6.0,32.0,39170.0,39170.0,32.0,34088587.0,-118437488.0,66214.0,0.0,4.0,2009.0,27422107.0,31038350.0,3616243.0,372142.72,145.435246
2952594,10944745,0.0,6.0,2.0,427079.0,427079.0,2.0,34174889.0,-118302938.0,599071.0,0.0,1.0,1940.0,52359000.0,81780945.0,29421945.0,910972.42,142.559136
735349,11642367,11.0,6.0,16.0,52503.0,52503.0,16.0,34073263.0,-118427307.0,200522.0,0.0,1.0,1990.0,63662444.0,90188462.0,26526018.0,1078101.87,134.837986
1938915,11467277,0.0,6.0,2.0,522511.0,1542.0,2.0,33958979.0,-118418970.0,7000.0,0.0,401.0,2008.0,141417470.0,151895087.0,10477617.0,1840935.06,122.302122


In [11]:
df_dropped1_sorted.nsmallest(10, 'Rank')

,parcelid,bedroomcnt,buildingqualitytypeid,calculatedbathnbr,calculatedfinishedsquarefeet,finishedsquarefeet12,fullbathcnt,latitude,longitude,lotsizesquarefeet,roomcnt,unitcnt,yearbuilt,structuretaxvaluedollarcnt,taxvaluedollarcnt,landtaxvaluedollarcnt,taxamount,Rank
2524517,11587660,0.0,11.0,1.0,685.0,685.0,1.0,33996664.0,-118478888.0,2896.0,0.0,1.0,2013.0,247201.0,1228569.0,981368.0,14739.02,-187.370429
213213,11657341,1.0,12.0,1.0,1020.0,1020.0,1.0,34055405.0,-118468449.0,16998.0,0.0,1.0,2013.0,376896.0,4705941.0,4329045.0,56288.74,-186.557445
2132647,11537242,0.0,11.0,1.0,974.0,974.0,1.0,33986000.0,-118446000.0,7000.0,0.0,1.0,2007.0,71200.0,89000.0,17800.0,1191.28,-186.285396
2370703,11538710,0.0,11.0,1.0,1110.0,1110.0,1.0,33988700.0,-118443000.0,39561.0,0.0,1.0,2008.0,317895.0,438977.0,121082.0,5346.03,-186.270962
223232,11538701,0.0,11.0,1.0,1070.0,1070.0,1.0,33988700.0,-118443000.0,39561.0,0.0,1.0,2008.0,309237.0,468871.0,159634.0,5700.62,-186.228282
1759758,11538699,0.0,11.0,1.0,1070.0,1070.0,1.0,33988700.0,-118443000.0,39561.0,0.0,1.0,2008.0,309237.0,472686.0,163449.0,5662.66,-186.224023
357457,11538739,0.0,11.0,1.0,1000.0,1000.0,1.0,33988700.0,-118443000.0,39561.0,0.0,1.0,2008.0,289065.0,479339.0,190274.0,5738.99,-186.223051
14430,11538727,0.0,11.0,1.0,1070.0,1070.0,1.0,33988700.0,-118443000.0,39561.0,0.0,1.0,2008.0,309237.0,478247.0,169010.0,5728.94,-186.214635
1655230,11538716,0.0,11.0,1.0,1070.0,1070.0,1.0,33988700.0,-118443000.0,39561.0,0.0,1.0,2008.0,309237.0,478247.0,169010.0,5728.94,-186.214635
29414,11538723,0.0,11.0,1.0,1110.0,1110.0,1.0,33988700.0,-118443000.0,39561.0,0.0,1.0,2008.0,320796.0,476067.0,155271.0,5704.66,-186.212474


## Task 2 (10%)

> - Define a house “pairwise distance function”, which measures the similarity of two properties.  Like a distance metric, similar pairs of very similar properties should be distance near zero, with distance increasing as the properties grow more dissimilar.  
- Experiment with your distance function, and write a one page discussion evaluating how well you think it worked.   Your function should include geographic as well as property-specific variables. 


In [18]:
def pairwise_distance(X,Y):
    X.values.reshape(-1, 1)
    Y.values.reshape(-1, 1)
    return pairwise.euclidean_distances(X, Y, Y_norm_squared=None, squared=False, X_norm_squared=None)

X1 = df_dropped1_scaled.loc[rank.argmax()]
Y1 = df_dropped1_scaled.loc[rank.argmin()]

In [17]:
X2 = df_dropped1_scaled.loc[0]
Y2 = df_dropped1_scaled.loc[1]
print(pairwise_distance(X2,Y2))

[[ 29497.36496706]]


## Task 3 (15%)

> - Using your distance function and an appropriate clustering algorithm, cluster the houses using your distance function into 10 to 100 classes, as you see best.   
- Present a dot-plot/map (with tiny dots colored to reflect the clustering) illustrating the clusters your method produced.    
- Write a one page discussion/analysis of what your clusters seem to be capturing, and how well they work 



## Task 4 (20%)

> - Identify at least one external data set which you can integrate into your price prediction analysis to make it better.  Perhaps it can be financial, such as the historical effects of interest rates, consumer confidence, etc. on housing prices.  Perhaps it can be geographic, like the crime rate, educational scores, income levels, etc.   
- Write a one page discussion/analysis on whether this data helps with the prediction tasks 




## Task 5 (20%)

> - Finally, build the best prediction model you can to solve the Zillow task.  Use any data, ideas, and approach that you like. Predict the logerror for instances at file “sample_submission.csv”.  
- Report the score/rank you get. 
- Write a 2-3 page report about how it works, an evaluation, and any interesting experiences along the way.  



## Task 6 (15%)

> - Do a permutation test to determine a p-value of how good your predictions of logerror are.  You can use whatever metric you wish to score your model (like mean absolute error).   
- For a large enough sample of the evaluation data, compare how your model ranks by this metric on the real data compared to 100 (or more) random permutations of the logerror assigned to the real data records. 
- What fraction of permutations produce at least error at least as good at the real data set?   If necessary, sample your data so these 100+ runs do not take too much time. 




## Task 7 (10%)

> Submit your results on the real test data to Kaggle before deadline. Write the result into a csv file and submit it to the website. Actually, submit two for your two best models, to the extent that Zillow allows it.   


